In [1]:
import os
import pandas as pd
from scipy.integrate import odeint
from scipy.interpolate import interp1d

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
import sys

sys.path.append("..")
from symbolic_RLC import fxu_ODE, fxu_ODE_mod, A_nominal, B_nominal
from torchid.ssfitter import  NeuralODE
from torchid.ssmodels import StateSpaceModelLin, NeuralStateSpaceModelLin

In [2]:
Ts = 2e-7
A_lin = A_nominal * Ts
B_lin = B_nominal * Ts

A_lin = np.array([[10, 100], [1000, 10000]])
#ss_model = StateSpaceModelLin(A_lin, B_lin)
ss_model = NeuralStateSpaceModelLin(A_lin, B_lin)


# In[Linearization time - 1 a time]
nx = 2
nu = 1

In [3]:
# Linearization variable
VAR = []
for idx_var in range(nx):
    var = np.zeros((1,nx)).astype(np.float32)
    var[0,idx_var] = 1.0 # differentiate w.r.t the nth variable
    VAR.append(torch.tensor(var))

In [4]:
# Random linearization point
x_arr = np.random.rand(nx).astype(np.float32)
x_torch = torch.tensor(x_arr, requires_grad=True)

u_batch = np.random.rand(nu).astype(np.float32)
u_torch = torch.tensor(u_batch, requires_grad=True)


In [5]:
%%timeit -n 1000

# Linearization around the random point 

F_xu = ss_model(x_torch,u_torch)
A = np.empty((nx,nx))
B = np.empty((nx,nu))

for idx_var in range(nx):
    var = VAR[idx_var]
    #var = np.zeros((1,nx)).astype(np.float32)
    #var[0,idx_var] = 1.0 # differentiate w.r.t the nth variable
    F_xu.backward(var, retain_graph=True)
    A[idx_var,:] = np.array(x_torch.grad)
    B[idx_var,:] = np.array(u_torch.grad)
    x_torch.grad.data.zero_()
    u_torch.grad.data.zero_()

490 µs ± 29.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
# Linearization around the random point 

F_xu = ss_model(x_torch,u_torch)
A = np.empty((nx,nx))
B = np.empty((nx,nu))

for idx_var in range(nx):
    var = VAR[idx_var]
    F_xu.backward(var, retain_graph=True)
    A[idx_var,:] = np.array(x_torch.grad)
    B[idx_var,:] = np.array(u_torch.grad)
    x_torch.grad.data.zero_()
    u_torch.grad.data.zero_()

In [7]:
print(A)
print(B)
#A[0,0]-10

[[9.99984646e+00 1.00000153e+02]
 [1.00000012e+03 1.00000000e+04]]
[[-0.0001949 ]
 [ 0.00527266]]


In [8]:
# Batch linearization variable
batch_size = 128
VAR = []
for idx_var in range(nx):
    var = np.zeros((batch_size,nx)).astype(np.float32)
    var[:,idx_var] = 1.0 # differentiate w.r.t the nth variable
    VAR.append(torch.tensor(var))

In [9]:
# Random batch of linearization point


x_batch = np.random.rand(batch_size, nx).astype(np.float32)
x_torch = torch.tensor(x_batch, requires_grad=True)

u_batch = np.random.rand(batch_size, nu).astype(np.float32)
u_torch = torch.tensor(u_batch, requires_grad=True)


In [10]:
%%timeit -n 1000

# In[Linearization time - batched]
    
F_xu = ss_model(x_torch,u_torch)
A = np.empty((batch_size,nx,nx))
B = np.empty((batch_size,nx,nu))
for idx_var in range(nx):
    var = VAR[idx_var]
    F_xu.backward(var, retain_graph=True)
    A[:,idx_var,:] = np.array(x_torch.grad)
    B[:,idx_var,:] = np.array(u_torch.grad)
    x_torch.grad.data.zero_()
    u_torch.grad.data.zero_()

570 µs ± 46.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
F_xu = ss_model(x_torch,u_torch)
A = np.empty((batch_size,nx,nx))
B = np.empty((batch_size,nx,nu))
for idx_var in range(nx):
    var = VAR[idx_var]
    F_xu.backward(var, retain_graph=True)
    A[:,idx_var,:] = np.array(x_torch.grad)
    B[:,idx_var,:] = np.array(u_torch.grad)
    x_torch.grad.data.zero_()
    u_torch.grad.data.zero_()

In [12]:
A

array([[[1.00000839e+01, 9.99998932e+01],
        [1.00000031e+03, 1.00000000e+04]],

       [[9.99983883e+00, 1.00000160e+02],
        [1.00000012e+03, 1.00000000e+04]],

       [[9.99987698e+00, 9.99999313e+01],
        [1.00000043e+03, 1.00000000e+04]],

       [[9.99973202e+00, 9.99996872e+01],
        [1.00000024e+03, 1.00000000e+04]],

       [[9.99992180e+00, 9.99996948e+01],
        [1.00000018e+03, 1.00000000e+04]],

       [[1.00001774e+01, 1.00000359e+02],
        [1.00000018e+03, 1.00000010e+04]],

       [[9.99985981e+00, 1.00000122e+02],
        [1.00000073e+03, 1.00000000e+04]],

       [[9.99977112e+00, 1.00000023e+02],
        [1.00000037e+03, 1.00000000e+04]],

       [[9.99969673e+00, 9.99999466e+01],
        [1.00000049e+03, 1.00000010e+04]],

       [[9.99996853e+00, 9.99998627e+01],
        [1.00000031e+03, 1.00000000e+04]],

       [[9.99977112e+00, 1.00000023e+02],
        [1.00000037e+03, 1.00000000e+04]],

       [[1.00000067e+01, 1.00000389e+02],
        [1.0